In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json

client_id = "M4g9rW2aSU0zNVXbp0Ve"
client_secret = "gfr6xh20HF"

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

def getPostData(post, jsonResult, cnt, target_year):
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    
    # Check if the year of the post matches the target year
    if pDate.year == target_year:
        title = post['title']
        description = post['description']
        org_link = post['originallink']
        link = post['link']
        
        pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
        jsonResult.append({'cnt': cnt, 'title': title, 'description': description,
                           'org_link': org_link, 'link': org_link, 'pDate': pDate})
        return True
    return False

def main():
    node = 'news'
    srcText = input('검색어를 입력하세요: ')
    year = int(input('검색을 원하는 년도를 입력하세요 (예: 2022): '))  # Ask for year
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            if getPostData(post, jsonResult, cnt, year):  # Modified function call
                cnt += 1

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)

    print('전체 검색 : %d 건' % total)

    with open('%s_naver_%s_%s.json' % (srcText, node, year), 'w', encoding='utf8') as outfile:  # Save file with year
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print('%s_naver_%s_%s.json SAVED' % (srcText, node, year))  # Modified print statement



In [8]:
main()

[2023-08-08 16:28:31.718772] Url Request Success
[2023-08-08 16:28:31.825266] Url Request Success
[2023-08-08 16:28:31.926936] Url Request Success
[2023-08-08 16:28:32.039566] Url Request Success
[2023-08-08 16:28:32.135756] Url Request Success
[2023-08-08 16:28:32.234007] Url Request Success
[2023-08-08 16:28:32.339684] Url Request Success
[2023-08-08 16:28:32.440950] Url Request Success
[2023-08-08 16:28:32.524398] Url Request Success
[2023-08-08 16:28:32.616505] Url Request Success
HTTP Error 400: Bad Request
[2023-08-08 16:28:32.706736] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%8B%9C%EB%94%94%EC%A6%88&start=1001&display=100
전체 검색 : 8862 건
가져온 데이터 : 0 건
시디즈_naver_news_2019.json SAVED
